# Project 2: Forest Fire Simulation

In this project, you will simulate the spread of a forest fire on a 100×100 grid. Each cell in the grid can be in one of four states: tree, grassland, burning tree, or burnt tree. The goal is to understand how forest density influences fire spread, and to explore extensions such as the effect of wind.

## Part 1: Initial Setup of the Grid
- Grid and States:
- Create a 100×100 NumPy array to represent the forest.
- Each cell can have one of the following values:
	0: Grassland (non-flammable)
	1: Tree
	2: Burning tree
	3: Burnt tree
- Random Initialization: 
	- Implement a function that initializes the grid with trees and grassland based on a tree density parameter (between 0 and 1).
	- Each cell should be a tree with probability = density, and grassland otherwise.
Lightning Strike:
	- Randomly select a cell that contains a tree (1) and set it to burning (2).

## Part 2: Fire Spread Simulation
- Simulation Step:
	- In each time step:
		- A burning tree (2) becomes a burnt tree (3).
		- All trees (1) that are directly adjacent (up, down, left, right) to a burning tree catch fire and become burning trees (2) in the next step.
- Run the Simulation:
	-Continue the simulation step-by-step until no more trees are burning.
- Track Results:
	- Record the number of trees burnt at the end of the simulation.
- Calculate the percentage of trees burnt compared to the initial number of trees.


## Part 3: Visualization and Analysis
- Graphical Representation:
	- Write a function that can visualize the grid at each time step by using matplotlib. Use different colors for each state.
	- Write a function that allows you to created an animated gif for a full run of the simulation, where each frame/picture corresponds to a time-step
- Density Curve:
	- Run the simulation for various density values (e.g., from 0.1 to 1.0 in steps of 0.05). Clearly you need to run the simulation several times for each density value and take then the mean value of the resulting percentage of trees burnt.
	- Plot the percentage of trees burnt as a function of the initial density including the statistical uncertainties.
- Critical Density:
	- From the plot, identify the critical density above which the fire spreads through almost the entire forest.
- Expert Challenge (not strictly required to be handed in, however is required for achieving the best mark): Larger grids, e.g. 1000x1000 require in a naive simulation significantly more time, hence it is advisable to think about certain optimization aspects. One promising approach is a clever usage of numpy arrays and slicing. You can add one additional "optimizedSimulation" function that can handle efficiently also larger map sizes and is more time-effective than a naive approach that directly loops on your arrays.

## Part 4: Extensions – Wind Effect
- Wind Influence:
	- Modify the fire-spread rule to account for wind blowing in one direction (e.g., east). For example, a tree that is east of a burning tree can catch fire even if it is not only the direct next neighbour. The strength of the wind might determine the spread radius in one direction. It is important that you come up here with your own model of how wind might effect the spread of fire. It is your task for explain the choice of your model. Think independently! As long as your model choice is sensible, you will get full points. 
Discuss:
	- What impact does wind have on the spread of the fire in your model?
	- How does the critical density change when wind is included?

## Deliverables:
- A Jupyter-Notebook implementing the simulation, including
	- A plot showing burnt tree percentage vs. initial density.
	- One Animation of the fire spreading at a given density and one lighting event.
	- A brief written summary of findings and observations about the critical density and wind effects.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def initialize (density, size = (100,100)):
    """Initializes grid representing the forest.

    Parameters
    ----------
    density: (float) probability that any given cell is a tree
    size: size of grid; default: (100, 100)
    
    Returns
    -------
    grid
        Numpy array of input size with values 0, 1, and 2. Exactly one cell has value 2.
    """
    grid = np.random.binomial(1, density, size = size)  # Set up grid

    trees = np.transpose(np.nonzero(grid))  # Extract indices where there are trees

    # Transpose is needed as nonzero() gives a tuple of arrays; 
    #       transpose() turns it into an array consisting of two-item lists

    lightning = np.random.choice(np.arange(trees[:,0].size))  # Select random tree
    grid[tuple(trees[lightning,:])] = 2  # Lightning strikes selected tree

    return grid

In [ ]:
def time_step (grid):
    """
    Moves time forward by one step
    Parameters
    ----------
    grid: numpy array representing the forest

    Returns
    -------
    grid: numpy array with updated values
    """
    width = grid[0,:].size  # Read off width of forest
    height = grid[:,0].size  # Read off height of forest

    burning_trees = np.transpose(np.where(grid==2))  # Find burning trees
    burning_trees = [tuple(ii) for ii in burning_trees]  # Turn indices into tuples to call elements

    # The following is done on all burning trees found in the forest
    for tree in burning_trees:
        grid[tree] = 3  # Change status to burnt tree
        x, y = tree  # Read off index of burnt tree

        # Indices of neighboring trees:
        left = (x-1, y)
        right = (x+1, y)
        up = (x, y-1)
        down = (x, y+1)


        # If a tree is not on some edge and adjacent cell is a tree, 
        #    change adjacent cell to a burning tree
        if x > 0 and grid[left]==1:
            grid[left] = 2

        if x < width - 1 and grid[right]==1:
            grid[right] = 2

        if y > 0 and grid[up]==1:
            grid[up] = 2

        if y < height - 1 and grid[down]==1:
            grid[down] = 2

    return grid

In [ ]:
def run_simulation(grid):
    """
    Runs forest fire simulation on the input initial condition until no tree is burning

    Parameters
    ----------
    grid: numpy array representing the forest's initial condition after lightning strike

    Returns
    -------
    grid: numpy array representing the forest after fire stops
    burnt: number of burnt trees
    """

    burning_trees = np.transpose(np.where(grid==2))

    while burning_trees.size > 0:
        grid = time_step(grid)

        burning_trees = np.transpose(np.where(grid==2))


    burnt_tress = (grid == 3)

    burnt = grid[burnt_tress].size

    return grid, burnt

In [4]:
grid = initialize(0.5, (10,10))

print(grid)

[[1 0 1 1 0 0 0 1 1 1]
 [0 0 0 0 1 0 0 0 0 1]
 [1 0 0 0 1 1 1 1 1 1]
 [1 1 1 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 0 1 1 0]
 [2 0 0 0 0 1 0 1 1 0]
 [0 1 0 0 1 1 1 0 1 0]
 [1 0 0 0 0 1 1 0 0 0]
 [1 1 0 0 1 1 1 0 0 1]
 [0 1 1 0 1 1 0 1 0 1]]


In [5]:
run_simulation(grid)

[[1 0 1 1 0 0 0 1 1 1]
 [0 0 0 0 1 0 0 0 0 1]
 [1 0 0 0 1 1 1 1 1 1]
 [1 1 1 0 0 0 0 0 0 0]
 [2 1 1 1 1 1 0 1 1 0]
 [3 0 0 0 0 1 0 1 1 0]
 [0 1 0 0 1 1 1 0 1 0]
 [1 0 0 0 0 1 1 0 0 0]
 [1 1 0 0 1 1 1 0 0 1]
 [0 1 1 0 1 1 0 1 0 1]]
[[1 0 1 1 0 0 0 1 1 1]
 [0 0 0 0 1 0 0 0 0 1]
 [1 0 0 0 1 1 1 1 1 1]
 [2 1 1 0 0 0 0 0 0 0]
 [3 2 1 1 1 1 0 1 1 0]
 [3 0 0 0 0 1 0 1 1 0]
 [0 1 0 0 1 1 1 0 1 0]
 [1 0 0 0 0 1 1 0 0 0]
 [1 1 0 0 1 1 1 0 0 1]
 [0 1 1 0 1 1 0 1 0 1]]
[[1 0 1 1 0 0 0 1 1 1]
 [0 0 0 0 1 0 0 0 0 1]
 [2 0 0 0 1 1 1 1 1 1]
 [3 2 1 0 0 0 0 0 0 0]
 [3 3 2 1 1 1 0 1 1 0]
 [3 0 0 0 0 1 0 1 1 0]
 [0 1 0 0 1 1 1 0 1 0]
 [1 0 0 0 0 1 1 0 0 0]
 [1 1 0 0 1 1 1 0 0 1]
 [0 1 1 0 1 1 0 1 0 1]]
[[1 0 1 1 0 0 0 1 1 1]
 [0 0 0 0 1 0 0 0 0 1]
 [3 0 0 0 1 1 1 1 1 1]
 [3 3 2 0 0 0 0 0 0 0]
 [3 3 3 2 1 1 0 1 1 0]
 [3 0 0 0 0 1 0 1 1 0]
 [0 1 0 0 1 1 1 0 1 0]
 [1 0 0 0 0 1 1 0 0 0]
 [1 1 0 0 1 1 1 0 0 1]
 [0 1 1 0 1 1 0 1 0 1]]
[[1 0 1 1 0 0 0 1 1 1]
 [0 0 0 0 1 0 0 0 0 1]
 [3 0 0 0 1 1 1 1 1 1]
 [3 3 3